# Your name: Krissy Wong
## Assignment Name: CA06 – kNN based Recommender Engine




##1. The Application

###### At scale, this would look like recommending products on Amazon, articles on Medium, movies on Netflix, or videos on YouTube. Although, we can be certain they all use more efficient means of making recommendations due to the enormous volume of data they process. However, we could replicate one of these recommender systems on a smaller scale using what we have learned here in this article. Let us build the core of a movies recommender system.

###### What question are we trying to answer?Given a movies data set, what are the 5 most similar movies to a movie query?

In [1]:
# Import packages 
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

##2. Data Source and Contents


In [2]:
# Mount dataset from Google Drive 
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [19]:
# Read datasets and look at the head(5) of the data 
movie_data = pd.read_csv('/gdrive/MyDrive/BSAN6070/CA06/movies_recommendation_data.csv')
movie_data.head(5)

,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
0,58,The Imitation Game,8.0,1,1,1,0,0,0,0,0
1,8,Ex Machina,7.7,0,1,0,0,0,1,0,0
2,46,A Beautiful Mind,8.2,1,1,0,0,0,0,0,0
3,62,Good Will Hunting,8.3,0,1,0,0,0,0,0,0
4,97,Forrest Gump,8.8,0,1,0,0,0,0,0,0


In [6]:
# Look at the tail of the dataset 
movie_data.tail(5)

,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
25,17,Thor: Ragnarok,7.9,0,0,0,1,0,0,0,0
26,12,Spirit: Stallion of the Cimarron,7.1,0,0,0,0,0,0,0,0
27,1,Hacksaw Ridge,8.2,1,1,0,0,0,0,1,0
28,86,12 Years a Slave,8.1,1,1,0,0,0,0,1,0
29,46,Queen of Katwe,7.4,1,1,0,0,0,0,0,0


##### Data Inspection and Quality Analysis 

In [8]:
# Describes the data according to count, mean, std, min, 25%, 50%, 75%, and max

movie_data.describe()

,Movie ID,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.0
mean,48.133333,7.696667,0.233333,0.600000,0.100000,0.100000,0.133333,0.100000,0.100000,0.0
std,29.288969,0.666169,0.430183,0.498273,0.305129,0.305129,0.345746,0.305129,0.305129,0.0
min,1.000000,5.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,27.750000,7.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,48.500000,7.750000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,64.250000,8.175000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,98.000000,8.800000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0


In [9]:
# Verify the shape of the data to see the columns and rows 
# The shape indicates that the dataset has a sample size of 30 and 11 is the dimension
movie_data.shape

(30, 11)

In [10]:
# Look at the count of records for each column in the dataset 

movie_data.count()

Movie ID       30
Movie Name     30
IMDB Rating    30
Biography      30
Drama          30
Thriller       30
Comedy         30
Crime          30
Mystery        30
History        30
Label          30
dtype: int64

In [11]:
# Check all the column types 

movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Movie ID     30 non-null     int64  
 1   Movie Name   30 non-null     object 
 2   IMDB Rating  30 non-null     float64
 3   Biography    30 non-null     int64  
 4   Drama        30 non-null     int64  
 5   Thriller     30 non-null     int64  
 6   Comedy       30 non-null     int64  
 7   Crime        30 non-null     int64  
 8   Mystery      30 non-null     int64  
 9   History      30 non-null     int64  
 10  Label        30 non-null     int64  
dtypes: float64(1), int64(9), object(1)
memory usage: 2.7+ KB


In [15]:
# Check for null values in the data 

movie_data.isnull().sum()

Movie ID       0
Movie Name     0
IMDB Rating    0
Biography      0
Drama          0
Thriller       0
Comedy         0
Crime          0
Mystery        0
History        0
Label          0
dtype: int64

In [16]:
# Check for duplicate values 

movie_data.duplicated().sum()

0

In [21]:
# Since the dataset is small (N < 30, D < 20), as described by the shape, we will
# need to use the Brute Force method. We will use the Euclidena Distance 
# which calculates the sum squared distance between two points in each column. 
# IMDB ratings range from 5.9, lowest, to 8.8, highest, in this dataset. 
# The range for the other columns range from 0 to 1. 
movie_data


,Movie ID,Movie Name,IMDB Rating,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Label
0,58,The Imitation Game,8.0,1,1,1,0,0,0,0,0
1,8,Ex Machina,7.7,0,1,0,0,0,1,0,0
2,46,A Beautiful Mind,8.2,1,1,0,0,0,0,0,0
3,62,Good Will Hunting,8.3,0,1,0,0,0,0,0,0
4,97,Forrest Gump,8.8,0,1,0,0,0,0,0,0
5,98,21,6.8,0,1,0,0,1,0,1,0
6,31,Gifted,7.6,0,1,0,0,0,0,0,0
7,3,Travelling Salesman,5.9,0,1,0,0,0,1,0,0
8,51,Avatar,7.9,0,0,0,0,0,0,0,0
9,47,The Karate Kid,7.2,0,1,0,0,0,0,0,0


##### Data Cleaning and Processing 

In [28]:
# Clean up the dataset to drop unnecessary columns 
movie_new = movie_data.drop(["Movie ID", "Movie Name", "Label"], axis = 1)

# Adjust IMDB ratings so it ranges from 0.0 to 1.0
movie_new["Adjusted IMDB Rating"] = movie_new["IMDB Rating"]/10

# Drop old IMDB Rating column since we already created a new column for adjusted values 
movie_new = movie_new.drop(["IMDB Rating"], axis = 1)

# Check to see the head of the dataset 
movie_new.head(5)

,Biography,Drama,Thriller,Comedy,Crime,Mystery,History,Adjusted IMDB Rating
0,1,1,1,0,0,0,0,0.80
1,0,1,0,0,0,1,0,0.77
2,1,1,0,0,0,0,0,0.82
3,0,1,0,0,0,0,0,0.83
4,0,1,0,0,0,0,0,0.88


##3. Building your own Recommender System

###### You are building your own movie recommendation website which uses your Recommendation Engine at the back-end. You are going to build this back-end Recommendation Engine. Imagine a user is navigating your recommendation website, and he/she encounters a movie named “The Post”. The user is not sure if he/she wants to watch it, but its genres intrigue the user; he/she is curious about other similar movies. The user scrolls down to the “More Like This” section to see what recommendations your recommendation website will make, and the back-end algorithmic gears begin to turn. Your website sends a request to its back-end for the 5 movies that are most similar to The Post. The backend has a recommendation data set exactly like ours. It begins by creating the row representation (better known as a feature vector) for The Post, then it runs a program similar to the one below to search for the 5 movies that are most similar to The Post, and finally sends the results back to the user at your website.

In [31]:
# Our objective is to find 5 similar movies to "The Post" using euclidean distance 

def euclidean_distance(point_1, point_2):
  sum_squared_distance = 0
  for i in range(len(point_1)):
    sum_squared_distance += (point_1[i] - point_2[i])** 2
  return sum_squared_distance**0.5

# Input genre information about the movie "The Post"
the_post = (1, 1, 0, 0, 0, 0, 1, 0.72)

###### Calculate the Euclidean Distance for The Post 

In [33]:
# Set distance and index 

distance = []
index = []

# Calculate and collect distances and movie indexes similar to The Post

for n in range (0, 30):
  distance_movie = euclidean_distance(the_post, movie_new.iloc[n,:])
  distance.append(distance_movie)
  index.append(n)

# Sort distance in ascending order for the top 5 movie recommendations 
movie_recs = pd.Series(distance, index = index).sort_values(ascending = True)[:5]

# Find the movie names for the top 5 movies most similar to The Post 
print("Based on your preferences, you may also like:")
movie_data.iloc[movie_recs.index, 1]


Based on your preferences, you may also like:


28    12 Years a Slave
27       Hacksaw Ridge
29      Queen of Katwe
16      The Wind Rises
2     A Beautiful Mind
Name: Movie Name, dtype: object